In [ ]:
# Copyright 2021 NVIDIA CORPORATION
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import glob
import pandas as pd
import numpy as np
import cudf
import cupy
import gc

from datetime import datetime

In [2]:
files = glob.glob('/raid/recsys2021_pre_1/*') + glob.glob('/raid/recsys2021_pre_2/*') + glob.glob('/raid/recsys2021_pre_3/*')

In [3]:
files

['/raid/recsys2021_pre_1/part-00185.parquet',
 '/raid/recsys2021_pre_1/part-00230.parquet',
 '/raid/recsys2021_pre_1/part-00054.parquet',
 '/raid/recsys2021_pre_1/part-00232.parquet',
 '/raid/recsys2021_pre_1/part-00007.parquet',
 '/raid/recsys2021_pre_1/part-00216.parquet',
 '/raid/recsys2021_pre_1/part-00068.parquet',
 '/raid/recsys2021_pre_1/part-00132.parquet',
 '/raid/recsys2021_pre_1/part-00126.parquet',
 '/raid/recsys2021_pre_1/part-00209.parquet',
 '/raid/recsys2021_pre_1/part-00219.parquet',
 '/raid/recsys2021_pre_1/part-00252.parquet',
 '/raid/recsys2021_pre_1/part-00113.parquet',
 '/raid/recsys2021_pre_1/part-00154.parquet',
 '/raid/recsys2021_pre_1/part-00205.parquet',
 '/raid/recsys2021_pre_1/part-00030.parquet',
 '/raid/recsys2021_pre_1/part-00086.parquet',
 '/raid/recsys2021_pre_1/part-00202.parquet',
 '/raid/recsys2021_pre_1/part-00080.parquet',
 '/raid/recsys2021_pre_1/part-00173.parquet',
 '/raid/recsys2021_pre_1/part-00111.parquet',
 '/raid/recsys2021_pre_1/part-0000

In [4]:
import os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"

In [5]:
import dask as dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess

In [6]:
cluster = LocalCUDACluster(local_directory='/raid/dask6/', device_memory_limit=0.5)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:41021 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 429.50 GB


In [7]:
ddf = dask_cudf.read_parquet(files, columns=['b_user_id'])

In [8]:
cdf = ddf['b_user_id'].value_counts().reset_index()
cdf.columns = ['b_user_id', 'count']
cdf.to_parquet('/raid/buserid2.parquet')

In [9]:
ddf = dask_cudf.read_parquet(files, columns=['a_user_id'])

In [10]:
cdf = ddf['a_user_id'].value_counts().reset_index()
cdf.columns = ['a_user_id', 'count']
cdf.to_parquet('/raid/auserid2.parquet')

In [11]:
client.close()

In [1]:
import cudf
import gc

In [2]:
df1 = cudf.read_parquet('/raid/auserid2.parquet')
df2 = cudf.read_parquet('/raid/buserid2.parquet')

In [3]:
df = df1.merge(df2, how='outer', left_on='a_user_id', right_on='b_user_id')

In [6]:
del df1, df2
gc.collect()

97

In [7]:
df.columns = ['a_user_id', 'count_a', 'b_user_id', 'count_b']

In [8]:
df['count_a'] = df['count_a'].fillna(0)
df['count_b'] = df['count_b'].fillna(0)
df['count'] = df['count_a'].fillna(0)+df['count_b'].fillna(0)

In [9]:
df['user_id'] = df['b_user_id'].copy()
df.loc[df['user_id'].isna(), 'user_id'] = df.loc[df['user_id'].isna(), 'a_user_id'].copy()

In [20]:
dftmp = df[((df['count_a']>8)|(df['count_b']>8))&((df['count_a']>2)&(df['count_b']>2))]

In [25]:
dftmp = dftmp[['user_id', 'co']].reset_index()
dftmp.columns = ['drop', 'user_id']
dftmp = dftmp.reset_index()

In [27]:
dftmp = dftmp.drop(['drop'], axis=1)

In [29]:
dftmp.columns = ['user_id_', 'user_id']

In [30]:
dftmp['user_id_'] = dftmp['user_id_']+1

In [7]:
ddf = dask_cudf.read_parquet(files, columns=['b_user_id'])

In [13]:
col = 'user_id'
cdf = ddf['a_user_id'].value_counts().reset_index()
cdf.columns = ['a_user_id', 'count']
cdf.to_parquet('/raid/auserid2.parquet')

In [8]:
cdf = ddf['b_user_id'].value_counts().reset_index()
cdf.columns = ['b_user_id', 'count']
cdf.to_parquet('/raid/buserid2.parquet')

In [10]:
client.close()

In [1]:
import cudf

In [5]:
df1 = cudf.read_parquet('/raid/auserid2.parquet')
df2 = cudf.read_parquet('/raid/buserid2.parquet')

In [16]:
(df1[df1['count']>10]['a_user_id'].isin(df2[df2['count']>10]['b_user_id'])).mean()

0.5165284299139445

In [17]:
(df2[df2['count']>10]['b_user_id'].isin(df1[df1['count']>10]['a_user_id'])).mean()

0.355176054488751

In [6]:
df = df1.merge(df2, how='outer', left_on='a_user_id', right_on='b_user_id')

In [7]:
del df1, df2

In [8]:
import gc
gc.collect()

51

In [9]:
df.columns = ['a_user_id', 'count_a', 'b_user_id', 'count_b']

In [10]:
df['count_a'] = df['count_a'].fillna(0)
df['count_b'] = df['count_b'].fillna(0)
df['count'] = df['count_a'].fillna(0)+df['count_b'].fillna(0)

In [12]:
df['user_id'] = df['b_user_id'].copy()

In [15]:
df['user_id'] = df['b_user_id'].copy()
df.loc[df['user_id'].isna(), 'user_id'] = df.loc[df['user_id'].isna(), 'a_user_id'].copy()

In [20]:
dftmp = df[((df['count_a']>10)|(df['count_b']>10))]

In [23]:
dftmp = dftmp[['user_id']].reset_index()

In [ ]:
dftmp.columns = ['user_id_', 'user_id']

In [ ]:
dftmp